In [29]:
import torch
import torchvision

import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

from torchinfo import summary

from py_scripts import data_setup, engine
from HelperFunctions import download_data, set_seeds, plot_loss_curves

In [30]:
data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

data_20_percent_path

[INFO] data\pizza_steak_sushi_20_percent directory exists, skipping download.


WindowsPath('data/pizza_steak_sushi_20_percent')

In [31]:
device = "cpu" #"cuda" if torch.cuda.is_available() else "cpu"
device 

'cpu'

In [32]:
train_dir = data_20_percent_path / "train"
test_dir = data_20_percent_path / "test"

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi_20_percent/train'),
 WindowsPath('data/pizza_steak_sushi_20_percent/test'))

In [33]:
def create_effnetb2_model(num_classes:int=3, # default output classes = 3 (pizza, steak, sushi)
                          seed:int=42):
  # 1, 2, 3 Create EffNetB2 pretrained weights, transforms and model
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights=weights).to(device)

  # 4. Freeze all layers in the base model
  for param in model.parameters():
    param.requires_grad = False

  # 5. Change classifier head with random seed for reproducibility
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes)
  ).to(device)

  return model, transforms

In [34]:
effnetb2, effnetb2_transforms = create_effnetb2_model(num_classes=3,
                                                      seed=42)
     

In [35]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_dataloaders(train_dir: str, test_dir: str, transform: transforms.Compose, batch_size: int):

    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True)

    return train_dataloader, test_dataloader, train_data, test_data

In [36]:
train_dataloader_effnetb2, test_dataloader_effnetb2, train_dataset, test_dataset = create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transform=effnetb2_transforms,
                                                                                                 batch_size=32)

In [37]:
len(train_dataloader_effnetb2), len(test_dataloader_effnetb2)

(15, 5)

In [38]:
from py_scripts import engine

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(params=effnetb2.parameters(),
                             lr=1e-3)

# Training function (engine.py)
effnetb2_results = engine.train_model_epochs(model=effnetb2,
                                train_dataloader=train_dataloader_effnetb2,
                                test_dataloader=test_dataloader_effnetb2,
                                epochs=5,
                                optimizer=optimizer,
                                loss_fn=loss_fn, 
                                writer = None,
                                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

TrainAccuracy:: 56.67%
TestAccuracy:: 94.09%


 20%|██        | 1/5 [00:16<01:05, 16.48s/it]

TrainAccuracy:: 79.79%
TestAccuracy:: 92.56%


 40%|████      | 2/5 [00:32<00:49, 16.47s/it]

...

 60%|██████    | 3/5 [00:49<00:33, 16.65s/it]

TrainAccuracy:: 88.12%
TestAccuracy:: 94.09%


 80%|████████  | 4/5 [01:06<00:16, 16.83s/it]

TrainAccuracy:: 91.04%
TestAccuracy:: 95.34%


100%|██████████| 5/5 [01:23<00:00, 16.79s/it]


In [39]:
from timeit import default_timer as timer

def predict(img) -> tuple[dict, float]:
  # Start a timer
  start_time = timer()

  # Transform the input image for use with EffNetB2
  img = effnetb2_transforms(img).unsqueeze(0) # unsqueeze = add batch dimension on 0th index

  # Put model into eval mode, make prediction
  effnetb2.eval()
  with torch.inference_mode():
    # Pass transformed image through the model and turn the prediction logits into probaiblities
    pred_probs = torch.softmax(effnetb2(img), dim=1)
    
  # Create a prediction label and prediction probability dictionary
  pred_labels_and_probs = {train_dataset.classes[i]: float(pred_probs[0][i]) for i in range(len(train_dataset.classes))}

  # Calculate pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)

  # Return pred dict and pred time
  return pred_labels_and_probs, pred_time

In [40]:
from pathlib import Path

# Get all test data paths
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_data_paths[:5]

[WindowsPath('data/pizza_steak_sushi_20_percent/test/pizza/1001116.jpg'),
 WindowsPath('data/pizza_steak_sushi_20_percent/test/pizza/1032754.jpg'),
 WindowsPath('data/pizza_steak_sushi_20_percent/test/pizza/1067986.jpg'),
 WindowsPath('data/pizza_steak_sushi_20_percent/test/pizza/129666.jpg'),
 WindowsPath('data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg')]

In [41]:
import random
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['data\\pizza_steak_sushi_20_percent\\test\\steak\\831681.jpg'],
 ['data\\pizza_steak_sushi_20_percent\\test\\steak\\2069289.jpg'],
 ['data\\pizza_steak_sushi_20_percent\\test\\sushi\\1383396.jpg']]

In [44]:
import gradio as gr

title = "FV Mini"

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title)
demo.launch() 


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
